# Quantum State Tomography

* **Last Updated:** Jan 24, 2019
* **Requires:**
  * qiskit-terra 0.7
  * qiskit-aer 0.1

In [7]:
# Needed for functions
import numpy as np
import time

# Import Qiskit classes
import qiskit 
from qiskit import QuantumRegister, QuantumCircuit, ClassicalRegister, Aer
from qiskit.quantum_info import state_fidelity

# Import Qiskit Aer Noise
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise.errors import *

# Tomography functions
import sys, os
sys.path.append(os.path.abspath(os.path.join('../qiskit_ignis')))
import tomography as tomo

## 2-Qubit state tomography Example

In [3]:
# Create a state preparation circuit
q2 = QuantumRegister(2)
bell = QuantumCircuit(q2)
bell.h(q2[0])
bell.cx(q2[0], q2[1])
print(bell.qasm())

job = qiskit.execute(bell, Aer.get_backend('statevector_simulator'))
psi_bell = job.result().get_statevector(bell)
print(psi_bell)

OPENQASM 2.0;
include "qelib1.inc";
qreg q0[2];
h q0[0];
cx q0[0],q0[1];

[0.70710678+0.j 0.        +0.j 0.        +0.j 0.70710678+0.j]


In [4]:
# Generate circuits and run on simulator
t = time.time()
qst_bell = tomo.state_tomography_circuits(bell, q2)
job = qiskit.execute(qst_bell, Aer.get_backend('qasm_simulator'), shots=5000)
print('Time taken:', time.time() - t)

# Extract tomography data so that countns are indexed by measurement configuration
# Note that the None labels are because this is state tomography instead of process tomography
# Process tomography would have the preparation state labels there

tomo_counts_bell = tomo.tomography_data(job.result(), qst_bell)
tomo_counts_bell

Time taken: 0.2506849765777588


{('X', 'X'): {'11': 2486, '00': 2514},
 ('X', 'Y'): {'01': 1225, '11': 1235, '10': 1256, '00': 1284},
 ('X', 'Z'): {'01': 1236, '11': 1242, '10': 1288, '00': 1234},
 ('Y', 'X'): {'01': 1283, '11': 1229, '10': 1261, '00': 1227},
 ('Y', 'Y'): {'01': 2461, '10': 2539},
 ('Y', 'Z'): {'01': 1347, '11': 1228, '10': 1165, '00': 1260},
 ('Z', 'X'): {'01': 1281, '11': 1233, '10': 1211, '00': 1275},
 ('Z', 'Y'): {'01': 1232, '11': 1280, '10': 1255, '00': 1233},
 ('Z', 'Z'): {'11': 2547, '00': 2453}}

In [5]:
# Generate fitter data and reconstruct density matrix
probs_bell, basis_matrix_bell = tomo.fitter_data(tomo_counts_bell)
rho_bell = tomo.state_cvx_fit(probs_bell, basis_matrix_bell)
F_bell = state_fidelity(psi_bell, rho_bell)
print('Fit Fidelity =', F_bell)

Fit Fidelity = 0.9999583056919802


## Including Calibration Error (Simulated)

Suppose we include T1 relaxation during measurement, but all other circuit operations are perfect.
Noise parameters:
* measurement time: 5 microseconds
* T_1: 50 microseconds
* Population of excited state: 5%

The assignment fidelity for this can be calculated from
* P(0|0) = 1 - P(1|0)
* P(1|0) = pop1 * exp(- t_meas / t_1)
* P(0|1) = (1 - pop1) * exp(- t_meas / t_1)
* P(1|1) = 1 - P(0|1)

the calibration error matrix is
M = [[P(0|0), P(0|1)], [P(1|0), P(1|1)]]

In [11]:
pop1 = 0.05
T1 = 50
T2 = 2 * T1
t_meas = 5

error = thermal_relaxation_error(T1, T2, t_meas, pop1)

noise = NoiseModel()
noise.add_all_qubit_quantum_error(error, 'measure')

# Generate circuits and run on simulator
job_cal = qiskit.execute(qst_bell, Aer.get_backend('qasm_simulator'),
                         shots=5000,
                         basis_gates=noise.basis_gates,
                         noise_model=noise)

# Extract tomography data so that countns are indexed by measurement configuration
tomo_counts_noise = tomo.tomography_data(job_cal.result(), qst_bell)
tomo_counts_noise

{('X', 'X'): {'01': 218, '11': 2107, '10': 245, '00': 2430},
 ('X', 'Y'): {'01': 1256, '11': 1007, '10': 1217, '00': 1520},
 ('X', 'Z'): {'01': 1228, '11': 1031, '10': 1258, '00': 1483},
 ('Y', 'X'): {'01': 1271, '11': 1023, '10': 1222, '00': 1484},
 ('Y', 'Y'): {'01': 2324, '11': 33, '10': 2226, '00': 417},
 ('Y', 'Z'): {'01': 1285, '11': 1024, '10': 1196, '00': 1495},
 ('Z', 'X'): {'01': 1313, '11': 1045, '10': 1192, '00': 1450},
 ('Z', 'Y'): {'01': 1196, '11': 1051, '10': 1268, '00': 1485},
 ('Z', 'Z'): {'01': 198, '11': 2054, '10': 198, '00': 2550}}

In [12]:
# Create Calibration matrix

P1_0 = pop1 * np.exp(-t_meas / T1)
P0_1 = (1 - pop1) * np.exp(-t_meas / T1)

# Single qubit calibration matrix
cal_mat_single = np.array([[1 - P1_0, P0_1], [P1_0, 1 - P0_1]])

# 2-qubit calibration matrix
cal_mat = np.kron(cal_mat_single, cal_mat_single)

In [20]:
cal_mat

array([[0.91156309, 0.82070584, 0.82070584, 0.7389045 ],
       [0.04319504, 0.13405229, 0.03888971, 0.12069104],
       [0.04319504, 0.03888971, 0.13405229, 0.12069104],
       [0.00204683, 0.00635216, 0.00635216, 0.01971341]])

In [13]:
# Fitting without calibration matrix
probs_noise, basis_matrix_noise = tomo.fitter_data(tomo_counts_noise)
rho_noise = tomo.state_cvx_fit(probs_noise, basis_matrix_noise)
print('Fitting without using calibration data')
print('Fit Fidelity =', state_fidelity(psi_bell, rho_noise))

Fitting without using calibration data
Fit Fidelity = 0.8681889118450377


In [14]:
# Fitting with calibration matrix by modifying POVMs
probs_noise_cal, basis_matrix_noise_cal = tomo.fitter_data(tomo_counts_noise,
                                                     calibration_matrix=cal_mat)
rho_noise_cal = tomo.state_cvx_fit(probs_noise_cal, basis_matrix_noise_cal)
print('Using calibration data to update basis matrix')
print('Fit Fidelity =', state_fidelity(psi_bell, rho_noise_cal))

Using calibration data to update basis matrix
Fit Fidelity = 0.9788749509551824


## Generating and fitting random states

We now test the functions on state generated by a circuit consiting of a layer of random single qubit unitaries u3

In [15]:
def random_u3_tomo(nq, shots):
    
    def rand_angles():
        return tuple(2 * np.pi * np.random.random(3) - np.pi)
    q = QuantumRegister(nq)
    circ = QuantumCircuit(q)
    for j in range(nq):
        circ.u3(*rand_angles(), q[j])
    job = qiskit.execute(circ, Aer.get_backend('statevector_simulator'))
    psi_rand = job.result().get_statevector(circ)
    
    qst_circs = tomo.state_tomography_circuits(circ, q)
    job = qiskit.execute(qst_circs, Aer.get_backend('qasm_simulator'),
                         shots=shots)
    tomo_counts = tomo.tomography_data(job.result(), qst_circs)
    probs, basis_matrix = tomo.fitter_data(tomo_counts, beta=0.5)
    
    rho_cvx = tomo.state_cvx_fit(probs, basis_matrix)
    rho_mle = tomo.state_mle_fit(probs, basis_matrix)
    
    print('F fit (CVX) =', state_fidelity(psi_rand, rho_cvx))
    print('F fit (MLE) =', state_fidelity(psi_rand, rho_mle))

In [16]:
for j in range(5):
    print('Random single-qubit unitaries: set {}'.format(j))
    random_u3_tomo(3, 5000)

Random single-qubit unitaries: set 0
F fit (CVX) = 0.9977859347729403
F fit (MLE) = 0.9976950574691672
Random single-qubit unitaries: set 1
F fit (CVX) = 0.999495298501456
F fit (MLE) = 0.99750613658546
Random single-qubit unitaries: set 2
F fit (CVX) = 0.9972536043567923
F fit (MLE) = 0.9984838104775382
Random single-qubit unitaries: set 3
F fit (CVX) = 0.9984982190640335
F fit (MLE) = 0.9970772439464521
Random single-qubit unitaries: set 4
F fit (CVX) = 0.9989056856839248
F fit (MLE) = 0.9958511474490492


## 5-Qubit Bell State

In [17]:
# Create a state preparation circuit
q5 = QuantumRegister(5)
bell5 = QuantumCircuit(q5)
bell5.h(q5[0])
for j in range(4):
    bell5.cx(q5[j], q5[j + 1])

# Get ideal output state
job = qiskit.execute(bell5, Aer.get_backend('statevector_simulator'))
psi_bell5 = job.result().get_statevector(bell5)

# Generate circuits and run on simulator
t = time.time()
qst_bell5 = tomo.state_tomography_circuits(bell5, q5)
job = qiskit.execute(qst_bell5, Aer.get_backend('qasm_simulator'), shots=5000)

# Extract tomography data so that countns are indexed by measurement configuration
tomo_counts_bell5 = tomo.tomography_data(job.result(), qst_bell5)
probs_bell5, basis_matrix_bell5 = tomo.fitter_data(tomo_counts_bell5)
print('Time taken:', time.time() - t)

Time taken: 8.785384893417358


In [18]:
t = time.time()
rho_mle_bell5 = tomo.state_mle_fit(probs_bell5, basis_matrix_bell5)
print('MLE Reconstruction')
print('Time taken:', time.time() - t)
print('Fit Fidelity:', state_fidelity(psi_bell5, rho_mle_bell5))

MLE Reconstruction
Time taken: 2.460785150527954
Fit Fidelity: 0.993255253328152


In [19]:
t = time.time()
rho_sdp_bell5 = tomo.state_cvx_fit(probs_bell5, basis_matrix_bell5, solver='CVXOPT')
print('SDP Reconstruction')
print('Time taken:', time.time() - t)
print('Fidelity:', state_fidelity(psi_bell5, rho_sdp_bell5))

SDP Reconstruction
Time taken: 53.021812438964844
Fidelity: 0.9999274425924916
